In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_data(input_path):
    # Load and return all datasets
    users = pd.read_csv(os.path.join(input_path, 'BX-NewBooksUsers.csv'))
    ratings = pd.read_csv(os.path.join(input_path, 'BX-Ratings.csv'))
    books_old = pd.read_csv(os.path.join(input_path, 'BX-Books.csv'))
    new_books = pd.read_csv(os.path.join(input_path, 'BX-NewBooks.csv'))
    books = pd.concat([books_old, new_books]).drop_duplicates(subset=["Book-Title"], keep='first')
    return users, ratings, books, books_old, new_books

def prepare_data(books):
    # Prepare book data by creating a tags column and vectorizing it
    books['tags'] = books['Book-Author'] + " " + books['Book-Title'] + " " + books['Book-Publisher'] + " " + books['Year-Of-Publication'].astype(str)
    cv = CountVectorizer(max_features=500, stop_words="english")
    vectors = cv.fit_transform(books["tags"]).toarray()
    return vectors

def find_similar_books(book_id, books, new_books, vectors):
    # Calculate similarity and find similar books
    similarity = cosine_similarity(vectors)
    try:
        book_idx = new_books.index[new_books['ISBN'] == book_id].tolist()[0]
        similar_books = sorted(list(enumerate(similarity[book_idx])), key=lambda x: x[1], reverse=True)[1:10]
        return similar_books
    except IndexError:
        print("Book ID not found in dataset.")
        return []

def recommend_books(user_id, book_id, ratings, books, books_old, similar_books):
    # Recommend books based on the cosine similarity scores
    print("\nRecommend Book\n")
    for idx, sim_score in similar_books:
        similar_isbn = books.iloc[idx]['ISBN']
        if similar_isbn in books['ISBN'].values:
            book_title = books.iloc[idx]['Book-Title']
            book_author = books.iloc[idx]['Book-Author']
            isbn_ratings = ratings[ratings["ISBN"] == similar_isbn]
            book_year = books.iloc[idx]['Year-Of-Publication']
            book_publisher = books.iloc[idx]['Book-Publisher']
            if not isbn_ratings.empty:
                print(f"{book_title} by {book_author} published in {book_year} by {book_publisher}")

def recommend(user_id, book_id, input_path):
    users, ratings, books, books_old, new_books = load_data(input_path)
    vectors = prepare_data(books)
    similar_books = find_similar_books(book_id, books, new_books, vectors)
    if similar_books:
        recommend_books(user_id, book_id, ratings, books, books_old, similar_books)

def main():
    input_path = '../../Data-Files/Raw-Files'
    recommend(276762, "0425163091", input_path)

main()



Recommend Book

Drawing on the Right Side of the Brain: A Course in Enhancing Creativity and Artistic Confidence by Betty Edwards published in 1989 by Putnam Pub Group
Too Much Too Soon by Jacqueline Briskin published in 1991 by Berkley Pub Group
The Hundred Secret Senses by Amy Tan published in 1995 by Putnam Pub Group
Southern Cross by Patricia Daniels Cornwell published in 1999 by Putnam Pub Group
The List by Steven Paul Martini published in 1997 by Putnam Pub Group
Friendship with God: An Uncommon Dialogue by Neale Donald Walsch published in 1999 by Putnam Publishing Group
That Camden Summer by Lavyrle Spencer published in 1996 by Putnam Pub Group
